In [ ]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import Conv2D, Dense, Dropout, Activation, Flatten, MaxPooling2D
from keras.optimizers import SGD

In [ ]:
SET50 = ['ADVANC', 'AOT', 'BANPU', 'BBL', 'BCP', 'BDMS',
        'BEM', 'BH', 'BJC', 'BTS', 'CENTEL', 'CPALL', 'CPF',
        'CPN', 'DTAC', 'EGCO', 'HMPRO', 'INTUCH', 
        'IRPC', 'KBANK', 'KCE', 'KKP', 'KTB', 'LH', 'MINT', 
        'PTT', 'PTTEP', 'ROBINS', 'SCB', 'SCC', 
        'TCAP', 'TISCO', 'TMB', 'TOP', 'TRUE', 'TU']

In [ ]:
lr = 0.1
epochs = 1000
batch_size = 100
optimizer = SGD(lr=lr)
activator = 'tanh'
loss_method = 'mse'
metrics = ['mae', 'accuracy']

In [ ]:
def load_stock_data(symbol, year_start, year_end):
    stock_data = []
    with open('./Data set/SET50_OHLC/' + symbol + '.BK.csv', 'r') as csv_file:
        file_data = csv.reader(csv_file, delimiter=',')
        file_data = list(file_data)[1:]
        ma = []
        for row in file_data:
            if row[1] is '':
                continue
            elif len(ma) < 30:
                ma.append([float(row[1]), float(row[2]), float(row[3]), float(row[4])])
            elif int(row[0][0:4]) >= year_start and int(row[0][0:4]) <= year_end:
                temp = [float(row[1]), float(row[2]), float(row[3]), float(row[4])]
                stock_data.append(find_diff_ma(ma, temp))
                del ma[0]
                ma.append(temp)
    return stock_data

def find_diff_ma(moving_averages, current_values):
    returns = []
    for i in range(len(moving_averages[0])):
        data = [moving_average[i] for moving_average in moving_averages]
        ma = np.sum(data) / 30
        diff_ma = current_values[i] - ma
        returns.append(diff_ma)
    return returns
        

In [ ]:
stock_datas = [ load_stock_data(x, 2008, 2017) for x in SET50 ]

min_count = len(stock_datas[0])
rounds = min_count - 30
train_index = int(rounds * 0.75)
validate_index = train_index + int(rounds * 0.05)

In [ ]:
def create_model():
    model = Sequential()
    model.add(Conv2D(64, (7, 1), input_shape=(len(stock_datas), 30, 4), padding='same'))
    model.add(Activation(activator))
    model.add(MaxPooling2D(pool_size = (2, 1), data_format= "channels_last"))
    model.add(Flatten())
    model.add(Dense(1024))  
    model.add(Activation(activator))
    model.add(Dense(256))  
    model.add(Activation(activator))
    model.add(Dropout(0.2))
    model.add(Dense(len(stock_datas) * 4))
    
    return model

In [ ]:
model = create_model()
model.summary()
model.compile(loss=loss_method, optimizer=optimizer, metrics=metrics)

In [ ]:
data = []
count = 0
index = 0
while index < rounds:
    temp = [i[index : index + 30] for i in stock_datas]
    data.append(temp)
    index += 1
    
x = np.asarray(data)
x_train = x[0: train_index].astype('float32')
x_validate = x[train_index: validate_index].astype('float32')
x_test = x[validate_index: rounds].astype('float32')

print(x_train.shape, x_validate.shape, x_test.shape)

In [ ]:
target = []
index = 0
while index < rounds:
    target_data = [stock[index + 30] for stock in stock_datas]
    temp = []
    for data in target_data:
        temp = temp + data
    target.append(temp)
    index += 1
y = np.asarray(target)
y_train = y[0: train_index].astype('float32')
y_validate = y[train_index: validate_index].astype('float32')
y_test = y[validate_index: rounds].astype('float32')

print(y_train.shape, y_validate.shape, y_test.shape)

In [ ]:
model.fit(x_train, y_train, validation_data = (x_validate, y_validate), epochs = epochs, batch_size = batch_size, verbose= 1)